In [1]:
from sklearn import neighbors
from astropy.table import Table
from astropy.io import fits
import numpy as np
import pickle

In [2]:
galzone = Table.read('DR7_galzones.dat', format =  'ascii' )
depth = np.array(galzone['depth'])

In [30]:
galaxy_depth = []
for i in range(np.max(depth)):
    galaxy_depth.append(np.argwhere(depth==i))
galaxy_depth=np.array(galaxy_depth)

In [2]:
t = Table.read('DR7_zobovoids.dat', format =  'ascii' )
x_voidv2 = np.array(t['x'])
y_voidv2  = np.array(t['y'])
z_voidv2  = np.array(t['z'])
radius_voidv2  = np.array(t['radius'])

num_voidv2  = x_voidv2.shape[0]
voidv2_xyz = np.concatenate((x_voidv2.reshape(num_voidv2 ,1),
                                 y_voidv2.reshape(num_voidv2 ,1),
                                 z_voidv2.reshape(num_voidv2 ,1)), axis=1)

In [8]:
cols = ['ra', 'dec','redshift']
galaxy_table = Table.read('data_flags_updated.dat', format =  'ascii',include_names=cols )

In [9]:
# galaxy_table = galaxy_table[table['redshift']< 0.1026]

In [10]:
h=1
DtoR = np.pi/180.
RtoD = 180./np.pi
c = 3e5

r_gal = c*galaxy_table['redshift'].data/(100*h)
ra = galaxy_table['ra'].data
dec = galaxy_table['dec'].data

In [12]:
ra_radian = ra*DtoR

dec_radian = dec*DtoR

x_gal = r_gal*np.cos(ra_radian)*np.cos(dec_radian)

y_gal = r_gal*np.sin(ra_radian)*np.cos(dec_radian)

z_gal = r_gal*np.sin(dec_radian)

num_gal = x_gal.shape[0]
    
gals_xyz = np.concatenate((x_gal.reshape(num_gal,1),
                                 y_gal.reshape(num_gal,1),
                                 z_gal.reshape(num_gal,1)), axis=1)

In [13]:
for i in np.linspace(0.1,2,100):
    tree = neighbors.KDTree(gals_xyz, metric = 'euclidean')
    ind =  tree.query_radius(voidv2_xyz, r =i*radius_voidv2)
    with open('galaxy_in_voidv2_indices'+str(i), 'wb') as ind_file:
        pickle.dump(ind, ind_file)


In [30]:
trial = [[0,0,0],[1,1,1]]
trial_test = [[0,0,2],[0,5,0],[5,1,0],[5,0,0]] 
trial = np.array(trial)
trial_test = np.array(trial_test)

tree = neighbors.KDTree(trial, metric = 'euclidean')
ind =  tree.query_radius(trial_test, r =5)
# print("distance to closest neighbour is: ", dist) # distance between entry of trial_test and closest neighbour in trial
print("index of closest neighbour", ind) # indices from trial that are within radius r of entry in trial_test

index of closest neighbour [array([0, 1]) array([0, 1]) array([1]) array([0, 1])]


In [16]:
import pickle
with open('galaxy_in_maximal_indices', 'wb') as ind_file:
    pickle.dump(ind, ind_file)